In [3]:
%pip install kagglehub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amandam1/healthy-vs-diseased-leaf-image-dataset")

print("Path to dataset files:", path)

c:\Users\Arjun\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 6.20G/6.20G [19:17<00:00, 5.76MB/s]  

Extracting files...


Path to dataset files: C:\Users\Arjun\.cache\kagglehub\datasets\amandam1\healthy-vs-diseased-leaf-image-dataset\versions\1


In [11]:

import os
import random
import shutil
from pathlib import Path

# Set the random seed for reproducibility
random.seed(42)

# Define the base path for the leaf images database and the output directory
base_path = Path("A Database of Leaf Images")
output_dir = Path("Sampled_Dataset")

# Ensure output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Initialize dataset list
dataset = []

# Function to sample 10 random images from a folder
def get_random_images(folder_path, num_images=10):
    all_images = list(folder_path.glob("*.jpg"))  # Adjust if image extensions differ (e.g., ".png")
    sampled_images = random.sample(all_images, min(num_images, len(all_images)))
    return sampled_images

# Iterate over each plant folder in the base path
for plant_folder in base_path.iterdir():
    if plant_folder.is_dir():  # Ensure it's a directory
        healthy_path = plant_folder / "healthy"
        diseased_path = plant_folder / "diseased"
        
        # Check if both subfolders exist before proceeding
        if healthy_path.exists() and diseased_path.exists():
            # Get 10 random images from each "healthy" and "diseased" folder
            healthy_images = get_random_images(healthy_path, 10)
            diseased_images = get_random_images(diseased_path, 10)
            
            # Add to dataset with labels (0 for healthy, 1 for diseased)
            dataset.extend([(img, 0) for img in healthy_images])
            dataset.extend([(img, 1) for img in diseased_images])

# Shuffle the entire dataset
random.shuffle(dataset)

# Save the images and labels
labels_file = output_dir / "labels.txt"
with labels_file.open("w") as f:
    for idx, (image_path, label) in enumerate(dataset):
        # Define new path for the image in the output directory
        new_image_path = output_dir / f"{idx}_{'healthy' if label == 0 else 'diseased'}.jpg"
        
        # Copy the image to the new path
        shutil.copy(image_path, new_image_path)
        
        # Write the label to the labels file
        f.write(f"{new_image_path.name},{label}\n")

print("Images and labels saved successfully.")


Images and labels saved successfully.
